# Mieczmik: Assignment 4

In [18]:
import numpy as np
import pandas as pd
import sqlite3
import os
import tempfile

In [19]:
Badges = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','Badges.csv.gz'), compression = "gzip")
Comments = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','Comments.csv.gz'), compression = "gzip")
PostLinks = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','PostLinks.csv.gz'), compression = "gzip")
Posts = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','Posts.csv.gz'), compression = "gzip")
Tags = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','Tags.csv.gz'), compression = "gzip")
Users = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','Users.csv.gz'), compression = "gzip")
Votes = pd.read_csv(os.path.join(os.getcwd(),'travel_exchange_com','Votes.csv.gz'), compression = "gzip")

In [20]:
baza = os.path.join(tempfile.mkdtemp(), 'baza.sqlite')
if os.path.isfile(baza): # if this file already exists...
    os.remove(baza)
    
conn = sqlite3.connect(baza)
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

Exercise 1. SQL solution

In [21]:
sql1=pd.read_sql_query("""
SELECT Posts.Title, RelatedTab.NumLinks
                    FROM
                        (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
                        FROM PostLinks
                        GROUP BY RelatedPostId) AS RelatedTab
                    JOIN Posts ON RelatedTab.PostId=Posts.Id
                    WHERE Posts.PostTypeId=1
                    ORDER BY NumLinks DESC
""", conn)

Exercise 1. Pandas package solution

In [22]:
#Selecting title from Posts
PostLinks[['RelatedPostId']]
#Counting number of occurences of RelatedPostId
sql1_1=pd.read_sql_query("""SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks FROM PostLinks GROUP BY RelatedPostId""", conn)
pd1_1=PostLinks[['RelatedPostId']].groupby('RelatedPostId').size().rename("NumLinks").reset_index()

pd1=pd1_1.merge(Posts.loc[Posts.PostTypeId==1], left_on='RelatedPostId',right_on='Id')[['Title','NumLinks']]\
.sort_values(by=['NumLinks'],ascending=False, kind='mergesort').reset_index(drop=True)

In [23]:
pd1.equals(sql1)

True

In [24]:
pd1

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
...,...,...
4867,Is Cairns the only direct destination from the...,1
4868,EasyJet - Is the return flight valid if I miss...,1
4869,Are CBP officers allowed to search and clone m...,1
4870,US B1/B2 VISA not approved,1


Exercise 2. SQL solution

In [25]:
sql2=pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)

Exercise 2. Pandas package solution

In [26]:
# 1 Columns from Users
UsersCol = Users[['Id', 'DisplayName', 'Age', 'Location']]

# 2 Columns from Posts
PostsCol = Posts.loc[Posts.PostTypeId==1][['OwnerUserId', 'FavoriteCount']].dropna(axis=0)\
.groupby(['OwnerUserId'])['FavoriteCount'].agg(['sum', 'max']).reset_index()\
.rename(columns={"sum": "FavoriteTotal", "max": "MostFavoriteQuestionLikes"})\
.merge(Posts[['OwnerUserId', 'FavoriteCount', 'Title']], left_on=['OwnerUserId', 'MostFavoriteQuestionLikes'],right_on=['OwnerUserId', 'FavoriteCount'],\
)[['OwnerUserId', 'FavoriteTotal', 'MostFavoriteQuestionLikes', 'Title']].rename(columns={'Title': 'MostFavoriteQuestion'})

pd2 = PostsCol.merge(UsersCol, left_on = 'OwnerUserId', right_on = 'Id').sort_values(by=['FavoriteTotal'],\
ascending=False, kind='mergesort')\
[['DisplayName', 'Age', 'Location', 'FavoriteTotal', 'MostFavoriteQuestion', 'MostFavoriteQuestionLikes']]\
.head(n=10).reset_index(drop=True)

In [27]:
pd2.equals(sql2)

True

In [28]:
pd2

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


Exercise 3. SQL solution

In [29]:
sql3=pd.read_sql_query("""
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
    SELECT
        PostID,
        UserID,
        SUM(Score) AS CommentsTotalScore
    FROM Comments
    GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
""", conn)

Exercise 3. Pandas package solution

In [30]:
#ComTotScr
ComTotScr = Comments[['UserId', 'PostId', 'Score']].groupby(['UserId', 'PostId']).sum()

pd3 = Posts.loc[Posts.PostTypeId==1][['Id', 'OwnerUserId', 'Title']].merge(ComTotScr, left_on=['Id', 'OwnerUserId'], \
right_on=['PostId', 'UserId']).rename(columns={'Score': 'CommentsTotalScore'}).sort_values(by=['CommentsTotalScore'],ascending=False, kind='mergesort').head(n=10).reset_index(drop=True)[['Title','CommentsTotalScore']]

In [31]:
pd3.equals(sql3)

True

In [32]:
pd3

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20
